In [1]:
import requests
import pandas as pd
from tqdm import tqdm
import os

def download_images(image_links, group_ids, save_dir):
    # Create a directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)

    # Iterate through each URL in the list and download the image
    for i, url in tqdm(enumerate(image_links)):
        try:
            # Use group_id as the image filename
            group_id = group_ids[i]
            image_name = f"{group_id}.jpg"  # Save with group_id as name, using .jpg extension
            image_path = os.path.join(save_dir, image_name)

            # Get the image from the URL
            response = requests.get(url, stream=True)
            if response.status_code == 200:
                with open(image_path, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)
            else:
                print(f"Failed to download image from {url}")
        except Exception as e:
            print(f"Error occurred for {url}: {e}")

# Example usage
train_df = pd.read_csv('dataset/test.csv')
image_dir = 'images'

# Call the function with image links and group ids
download_images(train_df['image_link'].tolist(), train_df['group_id'].tolist(), image_dir)


47072it [44:25,  6.23it/s]

Error occurred for https://m.media-amazon.com/images/I/51SpXWGC9GL.jpg: HTTPSConnectionPool(host='m.media-amazon.com', port=443): Max retries exceeded with url: /images/I/51SpXWGC9GL.jpg (Caused by SSLError(SSLError(1, '[SSL: TLSV1_ALERT_INTERNAL_ERROR] tlsv1 alert internal error (_ssl.c:1007)')))


50914it [48:04,  4.59it/s]

Error occurred for https://m.media-amazon.com/images/I/51X-TDHLzsL.jpg: HTTPSConnectionPool(host='m.media-amazon.com', port=443): Max retries exceeded with url: /images/I/51X-TDHLzsL.jpg (Caused by SSLError(SSLError(1, '[SSL: TLSV1_ALERT_INTERNAL_ERROR] tlsv1 alert internal error (_ssl.c:1007)')))


80503it [1:17:28, 15.87it/s]

Error occurred for https://m.media-amazon.com/images/I/610-TNUOrPL.jpg: [Errno 13] Permission denied: 'images\\970563.jpg'


112362it [1:53:08, 15.50it/s]

Error occurred for https://m.media-amazon.com/images/I/61l9kNwjFeL.jpg: [Errno 13] Permission denied: 'images\\892291.jpg'


131187it [2:21:00, 15.51it/s]


In [ ]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 20.7 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import numpy as np
import re
import cv2
from PIL import Image, ImageEnhance
import sys
import os
import re
# Correct the path based on the directory structure you extracted
sys.path.append('/content/student_resource 3/student_resource 3') # Fixed path
import easyocr
from tqdm import tqdm
from src.constants import entity_unit_map, allowed_units # Now this should work

# Initialize EasyOCR Reader
reader = easyocr.Reader(['en'], gpu=False)

# [Define your helper functions here as before]

# Download images
#print("Downloading images...")
#download_images(df['image_link'], save_dir=image_dir)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

In [ ]:
# Load train data
df = pd.read_csv('/content/student_resource 3/student_resource 3/dataset/train.csv', index_col=False) # Changed directory to the correct location

# Create 'index' column if missing
if 'index' not in df.columns:
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'index'}, inplace=True)

# Ensure output directory exists
image_dir = '/content/student_resource 3/student_resource 3/images' # Changed directory to the correct location
os.makedirs(image_dir, exist_ok=True)

In [ ]:
unit_mapping = {
    'g': 'gram',
    'grams': 'gram',
    'kgs': 'kilogram',
    'kg': 'kilogram',
    'kilograms': 'kilogram',
    'lbs': 'pound',
    'lb': 'pound',
    'oz': 'ounce',
    'ounces': 'ounce',
    'mg': 'milligram',
    'mcg': 'microgram',
    'cm': 'centimetre',
    'cms': 'centimetre',
    'mm': 'millimetre',
    'm': 'metre',
    'in': 'inch',
    'inches': 'inch',
    'ft': 'foot',
    'feet': 'foot',
    'yd': 'yard',
    'yards': 'yard',
    'kv': 'kilovolt',
    'mv': 'millivolt',
    'v': 'volt',
    'w': 'watt',
    'kw': 'kilowatt',
    'l': 'litre',
    'ml': 'millilitre',
    'liters': 'litre',
    'litres': 'litre',
    'cc': 'cubic centimetre',
    'cu ft': 'cubic foot',
    'cu in': 'cubic inch',
    # Add more mappings as necessary
}

# Conversion factors between units (unit_from, unit_to): factor
conversion_factors = {
    ('kilogram', 'gram'): 1000,
    ('pound', 'gram'): 453.592,
    ('ounce', 'gram'): 28.3495,
    ('milligram', 'gram'): 0.001,
    ('microgram', 'gram'): 1e-6,
    ('kilovolt', 'volt'): 1000,
    ('millivolt', 'volt'): 0.001,
    ('kilowatt', 'watt'): 1000,
    ('millilitre', 'litre'): 0.001,
    ('centimetre', 'metre'): 0.01,
    ('millimetre', 'metre'): 0.001,
    ('inch', 'metre'): 0.0254,
    ('foot', 'metre'): 0.3048,
    ('yard', 'metre'): 0.9144,
    # Add more conversion factors as needed
}

def preprocess_image(image_path):
    # Load image
    image = Image.open(image_path)
    # Convert to grayscale
    image = image.convert('L')
    # Enhance contrast
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(2)
    # Convert to OpenCV format
    image = np.array(image)
    # Apply thresholding
    _, image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # Return PIL image
    return Image.fromarray(image)

def extract_text(image):
    result = reader.readtext(np.array(image), detail=0)
    return ' '.join(result)

def extract_entity(text, entity_name):
    patterns = {
        'item_weight': r'(\d+(\.\d+)?)\s*(\w+)',
        'maximum_weight_recommendation': r'(\d+(\.\d+)?)\s*(\w+)',
        'width': r'(\d+(\.\d+)?)\s*(\w+)',
        'height': r'(\d+(\.\d+)?)\s*(\w+)',
        'depth': r'(\d+(\.\d+)?)\s*(\w+)',
        'voltage': r'(\d+(\.\d+)?)\s*(\w+)',
        'wattage': r'(\d+(\.\d+)?)\s*(\w+)',
        'item_volume': r'(\d+(\.\d+)?)\s*(\w+)',
    }
    pattern = patterns.get(entity_name)
    matches = re.findall(pattern, text, re.IGNORECASE)
    if matches:
        for match in matches:
            value = match[0]
            unit = match[2].lower()
            return value, unit
    return None, None

def normalize_unit(unit):
    unit = unit.lower()
    unit = unit.strip('.')
    if unit in unit_mapping:
        return unit_mapping[unit]
    else:
        return unit

def convert_unit(value, unit, entity_name):
    target_units = entity_unit_map[entity_name]
    if unit in target_units:
        return value, unit  # No conversion needed
    else:
        for target_unit in target_units:
            key = (unit, target_unit)
            if key in conversion_factors:
                value_in_target_unit = float(value) * conversion_factors[key]
                return str(value_in_target_unit), target_unit
        # If no conversion possible, return original
        return value, unit

def format_prediction(value, unit):
    value = float(value)
    formatted_value = f"{value:.2f}".rstrip('0').rstrip('.')
    prediction = f"{formatted_value} {unit}"
    return prediction

In [ ]:
predictions = []

print("Processing images...")
for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    index = row['index']
    image_link = row['image_link']
    entity_name = row['entity_name']
    image_filename = os.path.basename(image_link)
    image_path = os.path.join(image_dir, image_filename)

    if not os.path.exists(image_path):
        # Image not found, skip
        prediction = ''
        predictions.append({'index': index, 'prediction': prediction})
        continue

    # Preprocess image
    preprocessed_image = preprocess_image(image_path)

    # Extract text
    text = extract_text(preprocessed_image)

    # Extract entity value
    value, unit = extract_entity(text, entity_name)

    if value and unit:
        # Normalize unit
        unit = normalize_unit(unit)

        # Convert unit if necessary
        value, unit = convert_unit(value, unit, entity_name)

        # Validate unit
        if unit in entity_unit_map[entity_name]:
            prediction = format_prediction(value, unit)
        else:
            prediction = ''
    else:
        prediction = ''

    predictions.append({'index': index, 'prediction': prediction})

# Create DataFrame and save predictions
output_df = pd.DataFrame(predictions)
output_df.to_csv('train_predictions.csv', index=False)
print("Predictions saved to train_predictions.csv")


Processing images...


100%|██████████| 263859/263859 [00:27<00:00, 9687.69it/s] 


Predictions saved to train_predictions.csv


In [ ]:
# Create DataFrame and save predictions
output_df = pd.DataFrame(predictions)
output_df.to_csv('test_out.csv', index=False)
print("Predictions saved to test_out.csv")

Predictions saved to test_out.csv
